main_pon_0214のコピー

## <font color="Orange"> import libraries</font>

In [1]:
import csv
import datetime
import os
import pickle
import statistics
import time
import warnings
import re
import numpy as np
import pandas as pd
import pandas.tseries.offsets as offsets
import scipy.ndimage
import lightgbm as lgb
import xgboost as xgb
import pywt 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from keras.utils import np_utils
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping 
from keras.layers import Input, Conv2D, BatchNormalization, ReLU, Flatten, Dense, Add, Dropout, Activation, MaxPooling2D
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_rows', 200)

## <font color="Orange">  data loading</font>

 #### <font color="MediumTurquoise">  ap</font>

In [3]:
_df_ap_org_1 = pd.read_csv('./data/0209/ap_1.csv',  error_bad_lines=False)
_df_ap_1 = _df_ap_org_1.dropna()
_df_ap_1 = _df_ap_1[['CSI_DATA','timestamp']]

_df_ap_org_2 = pd.read_csv('./data/0209/ap_2.csv',  error_bad_lines=False)
_df_ap_2 = _df_ap_org_2.dropna()
_df_ap_2 = _df_ap_2[['CSI_DATA','timestamp']]

b'Skipping line 124130: expected 27 fields, saw 92\nSkipping line 124139: expected 27 fields, saw 51\nSkipping line 124179: expected 27 fields, saw 77\n'
b'Skipping line 140236: expected 27 fields, saw 52\n'
b'Skipping line 366679: expected 27 fields, saw 49\nSkipping line 367434: expected 27 fields, saw 52\nSkipping line 367438: expected 27 fields, saw 52\nSkipping line 367818: expected 27 fields, saw 52\nSkipping line 368443: expected 27 fields, saw 52\nSkipping line 368566: expected 27 fields, saw 52\nSkipping line 368661: expected 27 fields, saw 52\nSkipping line 368683: expected 27 fields, saw 52\nSkipping line 369544: expected 27 fields, saw 52\nSkipping line 369688: expected 27 fields, saw 52\nSkipping line 369689: expected 27 fields, saw 52\nSkipping line 369703: expected 27 fields, saw 52\nSkipping line 369705: expected 27 fields, saw 52\nSkipping line 369706: expected 27 fields, saw 52\nSkipping line 369707: expected 27 fields, saw 52\nSkipping line 369921: expected 27 fields

In [4]:
_df_ap_1.shape

(383826, 2)

In [5]:
_df_ap_1.head()

,CSI_DATA,timestamp
0,[118 96 7 0 2 15 0 17 -5 17 1 21 -1 22 -2 20 -...,1.612869e+09
1,[37 80 2 0 0 0 0 0 0 0 0 0 -10 12 -11 12 -12 1...,1.612869e+09
2,[37 80 2 0 0 0 0 0 0 0 0 0 -10 10 -11 10 -11 1...,1.612869e+09
3,[84 -64 4 0 0 0 0 0 0 0 0 0 12 -1 12 1 12 1 12...,1.612869e+09
4,[84 -64 4 0 0 0 0 0 0 0 0 0 -9 -5 -9 -5 -8 -6 ...,1.612869e+09


In [6]:
_df_ap_1['timestamp'] = _df_ap_1['timestamp'].astype('float64')
_df_ap_1['timestamp'] = list(map(datetime.datetime.fromtimestamp, _df_ap_1['timestamp']))
_df_ap_1['timestamp'] = _df_ap_1['timestamp'].round('1s')

_df_ap_2['timestamp'] = _df_ap_2['timestamp'].astype('float64')
_df_ap_2['timestamp'] = list(map(datetime.datetime.fromtimestamp, _df_ap_2['timestamp']))
_df_ap_2['timestamp'] = _df_ap_2['timestamp'].round('1s')

In [7]:
_df_ap_1.head()

,CSI_DATA,timestamp
0,[118 96 7 0 2 15 0 17 -5 17 1 21 -1 22 -2 20 -...,2021-02-09 20:17:52
1,[37 80 2 0 0 0 0 0 0 0 0 0 -10 12 -11 12 -12 1...,2021-02-09 20:17:59
2,[37 80 2 0 0 0 0 0 0 0 0 0 -10 10 -11 10 -11 1...,2021-02-09 20:17:59
3,[84 -64 4 0 0 0 0 0 0 0 0 0 12 -1 12 1 12 1 12...,2021-02-09 20:17:59
4,[84 -64 4 0 0 0 0 0 0 0 0 0 -9 -5 -9 -5 -8 -6 ...,2021-02-09 20:17:59


In [8]:
# _df_ap_1['timestamp'] = _df_ap_1['timestamp'] + offsets.Hour(9)
# _df_ap_2['timestamp'] = _df_ap_2['timestamp'] + offsets.Hour(9)

In [9]:
_df_ap_1.head()

,CSI_DATA,timestamp
0,[118 96 7 0 2 15 0 17 -5 17 1 21 -1 22 -2 20 -...,2021-02-09 20:17:52
1,[37 80 2 0 0 0 0 0 0 0 0 0 -10 12 -11 12 -12 1...,2021-02-09 20:17:59
2,[37 80 2 0 0 0 0 0 0 0 0 0 -10 10 -11 10 -11 1...,2021-02-09 20:17:59
3,[84 -64 4 0 0 0 0 0 0 0 0 0 12 -1 12 1 12 1 12...,2021-02-09 20:17:59
4,[84 -64 4 0 0 0 0 0 0 0 0 0 -9 -5 -9 -5 -8 -6 ...,2021-02-09 20:17:59


In [10]:
def csi_to_numpy(_x):
    _x = _x[1:-2]
    _x_box = re.split(' ', _x)
    
    if len(_x_box) != 128:
        return -999
    
    else:
        for _ in _x_box:
            try:
                _ = int(_)
            except:
                _ = 0
        return _x_box

In [11]:
_df_ap_1['CSI_DATA'] = _df_ap_1['CSI_DATA'].map(csi_to_numpy)
_df_ap_2['CSI_DATA'] = _df_ap_2['CSI_DATA'].map(csi_to_numpy)

In [12]:
print(_df_ap_1.shape)
print(_df_ap_2.shape)

(383826, 2)
(266575, 2)


In [13]:
_df_ap_1= _df_ap_1[_df_ap_1['CSI_DATA'] != -999]
_df_ap_2 = _df_ap_2[_df_ap_2['CSI_DATA'] != -999]

In [14]:
print(_df_ap_1.shape)
print(_df_ap_2.shape)

(383684, 2)
(266556, 2)


In [15]:
col_csi = ['csi_{}'.format(i+1) for i in range(128)]

_csi_np_1 = _df_ap_1['CSI_DATA'].values
_csi_np_2 = _df_ap_2['CSI_DATA'].values

_csi_ls_1 = []
for _ls in _csi_np_1:
    _ls_int = []
    for n in _ls:
        try:
            _ls_int.append(int(n))
        except:
            _ls_int.append(0)
            print('error : {}'.format(n))
    _csi_ls_1.append(_ls_int)
_csi_ls_1 = np.array(_csi_ls_1)

_csi_ls_2 = []
for _ls in _csi_np_2:
    _ls_int = []
    for n in _ls:
        try:
            _ls_int.append(int(n))
        except:
            _ls_int.append(0)
            print('error : {}'.format(n))
    _csi_ls_2.append(_ls_int)
_csi_ls_2 = np.array(_csi_ls_2)

_df_ap_1[col_csi] = _csi_ls_1
_df_ap_2[col_csi] = _csi_ls_2

error : 
error : 
error : 


In [16]:
_df_ap_1.head()

,CSI_DATA,timestamp,csi_1,csi_2,csi_3,csi_4,csi_5,csi_6,csi_7,csi_8,...,csi_119,csi_120,csi_121,csi_122,csi_123,csi_124,csi_125,csi_126,csi_127,csi_128
0,"[118, 96, 7, 0, 2, 15, 0, 17, -5, 17, 1, 21, -...",2021-02-09 20:17:52,118,96,7,0,2,15,0,17,...,-2,-3,-3,1,-4,2,-1,6,1,8
1,"[37, 80, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, -10, 12...",2021-02-09 20:17:59,37,80,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[37, 80, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, -10, 10...",2021-02-09 20:17:59,37,80,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, -1...",2021-02-09 20:17:59,84,-64,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -9, -5...",2021-02-09 20:17:59,84,-64,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
_df_ap_1_bysecond = _df_ap_1.groupby('timestamp',as_index=False).agg(np.mean)
_df_ap_2_bysecond = _df_ap_2.groupby('timestamp',as_index=False).agg(np.mean)

In [18]:
_df_ap_1_bysecond.head()

,timestamp,csi_1,csi_2,csi_3,csi_4,csi_5,csi_6,csi_7,csi_8,csi_9,...,csi_119,csi_120,csi_121,csi_122,csi_123,csi_124,csi_125,csi_126,csi_127,csi_128
0,2021-02-09 20:17:52,118.000000,96.000000,7.000,0.0,2.0,15.0,0.0,17.0,-5.0,...,-2.0,-3.0,-3.0,1.0,-4.0,2.0,-1.0,6.0,1.0,8.0
1,2021-02-09 20:17:59,81.062500,-55.000000,3.875,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-02-09 20:18:00,83.986928,-64.209150,4.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-02-09 20:18:01,83.988235,-64.188235,4.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-02-09 20:18:02,83.969697,-64.484848,4.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
print(_df_ap_1_bysecond.shape)
print(_df_ap_2_bysecond.shape)

(2661, 129)
(1853, 129)


In [20]:
_df_ap = _df_ap_1_bysecond.append(_df_ap_2_bysecond, ignore_index=True)

In [21]:
_df_ap.shape

(4514, 129)

 #### <font color="MediumTurquoise">  mp</font>

In [22]:
_df_mp_org_1 = pd.read_csv('./data/0209/mp_3.csv',  error_bad_lines=False)
_df_mp_1 = _df_mp_org_1.dropna()
_df_mp_1 = _df_mp_1[['CSI_DATA','timestamp']]

_df_mp_org_2 = pd.read_csv('./data/0209/mp_4.csv',  error_bad_lines=False)
_df_mp_2 = _df_mp_org_2.dropna()
_df_mp_2 = _df_mp_2[['CSI_DATA','timestamp']]

_df_mp_org_3 = pd.read_csv('./data/0209/mp_5.csv',  error_bad_lines=False)
_df_mp_3 = _df_mp_org_3.dropna()
_df_mp_3 = _df_mp_3[['CSI_DATA','timestamp']]

b'Skipping line 144558: expected 27 fields, saw 49\nSkipping line 158698: expected 27 fields, saw 44\n'
b'Skipping line 213046: expected 27 fields, saw 52\n'
b'Skipping line 237302: expected 27 fields, saw 29\n'
b'Skipping line 290413: expected 27 fields, saw 44\nSkipping line 290614: expected 27 fields, saw 52\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
_df_mp_1['timestamp'] = _df_mp_1['timestamp'].astype('float64')
_df_mp_1['timestamp'] = list(map(datetime.datetime.fromtimestamp, _df_mp_1['timestamp']))
_df_mp_1['timestamp'] = _df_mp_1['timestamp'].round('1s')

_df_mp_2['timestamp'] = _df_mp_2['timestamp'].astype('float64')
_df_mp_2['timestamp'] = list(map(datetime.datetime.fromtimestamp, _df_mp_2['timestamp']))
_df_mp_2['timestamp'] = _df_mp_2['timestamp'].round('1s')

_df_mp_3['timestamp'] = _df_mp_3['timestamp'].astype('float64')
_df_mp_3['timestamp'] = list(map(datetime.datetime.fromtimestamp, _df_mp_3['timestamp']))
_df_mp_3['timestamp'] = _df_mp_3['timestamp'].round('1s')

In [24]:
# _df_mp_1['timestamp'] = _df_mp_1['timestamp'] + offsets.Hour(9)
# _df_mp_2['timestamp'] = _df_mp_2['timestamp'] + offsets.Hour(9)
# _df_mp_3['timestamp'] = _df_mp_3['timestamp'] + offsets.Hour(9)

In [25]:
_df_mp_1['CSI_DATA'] = _df_mp_1['CSI_DATA'].map(csi_to_numpy)
_df_mp_2['CSI_DATA'] = _df_mp_2['CSI_DATA'].map(csi_to_numpy)
_df_mp_3['CSI_DATA'] = _df_mp_3['CSI_DATA'].map(csi_to_numpy)

In [26]:
print(_df_mp_1.shape)
print(_df_mp_2.shape)
print(_df_mp_3.shape)

(93920, 2)
(333221, 2)
(199740, 2)


In [27]:
_df_mp_1= _df_mp_1[_df_mp_1['CSI_DATA'] != -999]
_df_mp_2 = _df_mp_2[_df_mp_2['CSI_DATA'] != -999]
_df_mp_3 = _df_mp_3[_df_mp_3['CSI_DATA'] != -999]

In [28]:
print(_df_mp_1.shape)
print(_df_mp_2.shape)
print(_df_mp_3.shape)

(93917, 2)
(333213, 2)
(199740, 2)


In [29]:
col_csi = ['csi_{}'.format(i+1) for i in range(128)]

_csi_np_1 = _df_mp_1['CSI_DATA'].values
_csi_np_2 = _df_mp_2['CSI_DATA'].values
_csi_np_3 = _df_mp_3['CSI_DATA'].values

_csi_ls_1 = []
for _ls in _csi_np_1:
    _ls_int = []
    for n in _ls:
        try:
            _ls_int.append(int(n))
        except:
            _ls_int.append(0)
            print('error : {}'.format(n))
    _csi_ls_1.append(_ls_int)
_csi_ls_1 = np.array(_csi_ls_1)

_csi_ls_2 = []
for _ls in _csi_np_2:
    _ls_int = []
    for n in _ls:
        try:
            _ls_int.append(int(n))
        except:
            _ls_int.append(0)
            print('error : {}'.format(n))
    _csi_ls_2.append(_ls_int)
_csi_ls_2 = np.array(_csi_ls_2)

_csi_ls_3 = []
for _ls in _csi_np_3:
    _ls_int = []
    for n in _ls:
        try:
            _ls_int.append(int(n))
        except:
            _ls_int.append(0)
            print('error : {}'.format(n))
    _csi_ls_3.append(_ls_int)
_csi_ls_3 = np.array(_csi_ls_3)

_df_mp_1[col_csi] = _csi_ls_1
_df_mp_2[col_csi] = _csi_ls_2
_df_mp_3[col_csi] = _csi_ls_3

In [30]:
_df_mp_1.head()

,CSI_DATA,timestamp,csi_1,csi_2,csi_3,csi_4,csi_5,csi_6,csi_7,csi_8,...,csi_119,csi_120,csi_121,csi_122,csi_123,csi_124,csi_125,csi_126,csi_127,csi_128
0,"[34, 32, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 33,...",2021-02-09 20:33:18,34,32,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[-125, 48, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 2...",2021-02-09 20:33:18,-125,48,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[-121, 112, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, -17,...",2021-02-09 20:33:18,-121,112,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[-17, -16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, ...",2021-02-09 20:33:18,-17,-16,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[82, -96, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 33,...",2021-02-09 20:33:19,82,-96,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
_df_mp_1_bysecond = _df_mp_1.groupby('timestamp',as_index=False).agg(np.mean)
_df_mp_2_bysecond = _df_mp_2.groupby('timestamp',as_index=False).agg(np.mean)
_df_mp_3_bysecond = _df_mp_3.groupby('timestamp',as_index=False).agg(np.mean)

In [32]:
_df_mp_1_bysecond.head()

,timestamp,csi_1,csi_2,csi_3,csi_4,csi_5,csi_6,csi_7,csi_8,csi_9,...,csi_119,csi_120,csi_121,csi_122,csi_123,csi_124,csi_125,csi_126,csi_127,csi_128
0,2021-02-09 20:33:18,-57.250000,44.000000,8.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-02-09 20:33:19,108.373832,93.906542,5.906542,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-02-09 20:33:20,110.000000,96.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-02-09 20:33:21,110.000000,96.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-02-09 20:33:22,110.000000,96.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
print(_df_mp_1_bysecond.shape)
print(_df_mp_2_bysecond.shape)
print(_df_mp_3_bysecond.shape)

(750, 129)
(2673, 129)
(1575, 129)


In [34]:
_df_mp_tmp = _df_mp_1_bysecond.append(_df_mp_2_bysecond, ignore_index=True)
_df_mp = _df_mp_tmp.append(_df_mp_3_bysecond, ignore_index=True)

In [35]:
_df_mp.shape

(4998, 129)

 #### <font color="MediumTurquoise">  label</font>

In [36]:
start_timestamp = pd.to_datetime('2021-02-09 00:00:00')

timestamp_box = [start_timestamp]

ts = start_timestamp
for _ in range(6653):
    ts = ts + offsets.Second(1)
    timestamp_box.append(ts)

timestamp_box = np.array(timestamp_box)
_df_timestamp = pd.DataFrame(timestamp_box, columns=['timestamp'])
_df_timestamp['label'] = 0

In [37]:
_df_timestamp.head()

,timestamp,label
0,2021-02-09 00:00:00,0
1,2021-02-09 00:00:01,0
2,2021-02-09 00:00:02,0
3,2021-02-09 00:00:03,0
4,2021-02-09 00:00:04,0


In [38]:
_df_timestamp.shape

(6654, 2)

In [39]:
_df_label = pd.read_csv('./data/0209/label.csv',  error_bad_lines=False)

In [40]:
_df_label['delta_time'] = _df_label['hour']*3600 +_df_label['minute']*60 + _df_label['second']

_df_np = _df_label[['label','delta_time']].values
label_box = []
for i in range(_df_np.shape[0]):
    try:
        n =_df_np[i+1][1]-_df_np[i][1]
        for _ in range(n):
            label_box.append(_df_np[i][0])
    except IndexError:
        break
        
label_box = np.array(label_box)

major_label_box = []
for _ in label_box:
    if 0 <= _ < 2:
        major_label_box.append(0)
    elif 2<= _ < 5:
        major_label_box.append(1)
    elif 5<= _ < 7:
        major_label_box.append(2)
major_label_box = np.array(major_label_box)

In [41]:
major_label_box

array([2, 2, 2, ..., 0, 0, 0])

In [42]:
_df_timestamp['label'] = label_box
_df_timestamp['major_label'] = major_label_box

In [43]:
_df_timestamp.head()

,timestamp,label,major_label
0,2021-02-09 00:00:00,6,2
1,2021-02-09 00:00:01,6,2
2,2021-02-09 00:00:02,6,2
3,2021-02-09 00:00:03,6,2
4,2021-02-09 00:00:04,6,2


In [44]:
_df_timestamp['timestamp'] = _df_timestamp['timestamp'] + offsets.Hour(20)
_df_timestamp['timestamp'] = _df_timestamp['timestamp'] + offsets.Minute(20)
_df_timestamp['timestamp'] = _df_timestamp['timestamp'] + offsets.Second(17)

In [45]:
_df_timestamp.head()

,timestamp,label,major_label
0,2021-02-09 20:20:17,6,2
1,2021-02-09 20:20:18,6,2
2,2021-02-09 20:20:19,6,2
3,2021-02-09 20:20:20,6,2
4,2021-02-09 20:20:21,6,2


 #### <font color="MediumTurquoise">  merge</font>

In [46]:
_df_ap = pd.merge(_df_ap, _df_timestamp, on=['timestamp'])
_df_mp = pd.merge(_df_mp, _df_timestamp, on=['timestamp'])

In [47]:
_df_ap.head()

,timestamp,csi_1,csi_2,csi_3,csi_4,csi_5,csi_6,csi_7,csi_8,csi_9,...,csi_121,csi_122,csi_123,csi_124,csi_125,csi_126,csi_127,csi_128,label,major_label
0,2021-02-09 20:20:17,84.0,-64.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2
1,2021-02-09 20:20:18,84.0,-64.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2
2,2021-02-09 20:20:19,84.0,-64.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2
3,2021-02-09 20:20:20,84.0,-64.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2
4,2021-02-09 20:20:21,84.0,-64.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2


In [48]:
_df_mp.head()

,timestamp,csi_1,csi_2,csi_3,csi_4,csi_5,csi_6,csi_7,csi_8,csi_9,...,csi_121,csi_122,csi_123,csi_124,csi_125,csi_126,csi_127,csi_128,label,major_label
0,2021-02-09 20:33:18,-57.250000,44.000000,8.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
1,2021-02-09 20:33:19,108.373832,93.906542,5.906542,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
2,2021-02-09 20:33:20,110.000000,96.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
3,2021-02-09 20:33:21,110.000000,96.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
4,2021-02-09 20:33:22,110.000000,96.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1


In [49]:
print(_df_ap.shape)
print(_df_mp.shape)

(4370, 131)
(4998, 131)


## <font color="Orange">  preprocessing</font>

 #### <font color="MediumTurquoise"> re-index</font>

In [50]:
col_reindex = ['timestamp', 'label', 'major_label']
for _ in range(64):
    col_reindex.append('csi_{}'.format(_+1))
    col_reindex.append('csi_{}'.format(_+65))
    
_df_ap = _df_ap.reindex(columns=col_reindex)
_df_mp = _df_mp.reindex(columns=col_reindex)

In [51]:
_df_ap.head()

,timestamp,label,major_label,csi_1,csi_65,csi_2,csi_66,csi_3,csi_67,csi_4,...,csi_60,csi_124,csi_61,csi_125,csi_62,csi_126,csi_63,csi_127,csi_64,csi_128
0,2021-02-09 20:20:17,6,2,84.0,0.0,-64.0,0.0,4.0,-0.145985,0.0,...,-0.080292,0.0,-0.350365,0.0,0.065693,0.0,-0.102190,0.0,-0.014599,0.0
1,2021-02-09 20:20:18,6,2,84.0,0.0,-64.0,0.0,4.0,-0.563291,0.0,...,-0.493671,0.0,-0.563291,0.0,-0.493671,0.0,-0.506329,0.0,-0.544304,0.0
2,2021-02-09 20:20:19,6,2,84.0,0.0,-64.0,0.0,4.0,0.073333,0.0,...,-0.453333,0.0,0.020000,0.0,-0.433333,0.0,0.120000,0.0,-0.466667,0.0
3,2021-02-09 20:20:20,6,2,84.0,0.0,-64.0,0.0,4.0,-0.557047,0.0,...,0.221477,0.0,-0.744966,0.0,0.154362,0.0,-0.570470,0.0,0.107383,0.0
4,2021-02-09 20:20:21,6,2,84.0,0.0,-64.0,0.0,4.0,0.097902,0.0,...,-0.048951,0.0,0.181818,0.0,-0.111888,0.0,0.237762,0.0,-0.237762,0.0


 #### <font color="MediumTurquoise"> FFT</font>

In [52]:
def _fft_and_filtering(_df):
    _df_result = _df
    
    # set cut-off-frequency
    cut_off_frequency = 20
    
    _csi_columns = []
    for _ in range(128):
        _csi_columns.append('csi_{}'.format(_+1))
    
    for col in _csi_columns:
        _data = _df[col].values
        
        # set parameters for fft
        N = len(_data) # data length
        dt = 0.01 # sampling interval
        fc = cut_off_frequency # cut-off frequency
        t = np.arange(0, N*dt, dt) # time-axis
        freq = np.linspace(0, 1.0/dt, N) # frequency-axis

        # fft 
        FFT = np.fft.fft(_data)
        FFT = FFT/(N/2)
        FFT[0] = FFT[0]/2
        FFT_copy = FFT.copy()
        FFT_copy[(freq < 0)] = 0
        FFT_copy[(freq >20)] = 0
        
        # inverse fft
        IFFT = np.fft.ifft(FFT_copy)
        IFFT = np.real(IFFT*N)

        col_modified = '{}_modified'.format(col)
        _df_result['{}_modified'.format(col)] = IFFT + np.mean(_data)
    
    return _df_result

In [53]:
_df_ap_modified = _fft_and_filtering(_df_ap)
_df_mp_modified = _fft_and_filtering(_df_mp)

In [54]:
_df_ap_modified.head()

,timestamp,label,major_label,csi_1,csi_65,csi_2,csi_66,csi_3,csi_67,csi_4,...,csi_119_modified,csi_120_modified,csi_121_modified,csi_122_modified,csi_123_modified,csi_124_modified,csi_125_modified,csi_126_modified,csi_127_modified,csi_128_modified
0,2021-02-09 20:20:17,6,2,84.0,0.0,-64.0,0.0,4.0,-0.145985,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-02-09 20:20:18,6,2,84.0,0.0,-64.0,0.0,4.0,-0.563291,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-02-09 20:20:19,6,2,84.0,0.0,-64.0,0.0,4.0,0.073333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-02-09 20:20:20,6,2,84.0,0.0,-64.0,0.0,4.0,-0.557047,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-02-09 20:20:21,6,2,84.0,0.0,-64.0,0.0,4.0,0.097902,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
col_reindex_modified = ['timestamp', 'label', 'major_label']
for _ in range(64):
    col_reindex_modified.append('csi_{}_modified'.format(_+1))
    col_reindex_modified.append('csi_{}_modified'.format(_+65))

_df_ap_modified = _df_ap_modified.reindex(columns=col_reindex_modified)
_df_mp_modified = _df_mp_modified.reindex(columns=col_reindex_modified)

In [56]:
_df_ap_modified.head()

,timestamp,label,major_label,csi_1_modified,csi_65_modified,csi_2_modified,csi_66_modified,csi_3_modified,csi_67_modified,csi_4_modified,...,csi_60_modified,csi_124_modified,csi_61_modified,csi_125_modified,csi_62_modified,csi_126_modified,csi_63_modified,csi_127_modified,csi_64_modified,csi_128_modified
0,2021-02-09 20:20:17,6,2,167.992156,0.0,-127.959596,-0.000020,7.999729,-1.131044,0.0,...,-0.917177,0.0,-1.232682,0.0,-0.863335,0.0,-1.066720,0.0,-0.914494,0.0
1,2021-02-09 20:20:18,6,2,167.999064,0.0,-127.984077,0.000002,8.000059,-0.672762,0.0,...,-0.521773,0.0,-0.804484,0.0,-0.458661,0.0,-0.611511,0.0,-0.556192,0.0
2,2021-02-09 20:20:19,6,2,168.002517,0.0,-127.995406,0.000068,8.000283,-0.443562,0.0,...,-0.563290,0.0,-0.523970,0.0,-0.552255,0.0,-0.379937,0.0,-0.643107,0.0
3,2021-02-09 20:20:20,6,2,167.997714,0.0,-127.978172,0.000087,8.000089,-0.640238,0.0,...,-0.949291,0.0,-0.621608,0.0,-0.998381,0.0,-0.563410,0.0,-1.046538,0.0
4,2021-02-09 20:20:21,6,2,167.991330,0.0,-127.955708,0.000032,7.999747,-1.091470,0.0,...,-1.244237,0.0,-1.022806,0.0,-1.304998,0.0,-1.013285,0.0,-1.323262,0.0


In [57]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='db1', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    return pywt.waverec(coeff, wavelet, mode='per')

In [58]:
def _dwt_and_filtering(_df):
    _df_result = _df
    
    # set cut-off-frequency
    cut_off_frequency = 20
    
    _csi_columns = []
    for _ in range(128):
        _csi_columns.append('csi_{}'.format(_+1))
    
    for col in _csi_columns:
        _data = _df[col].values
        
        _data_result = denoise(_data)

        col_modified = '{}_modified'.format(col)
        _df_result['{}_modified'.format(col)] = _data_result
    
    return _df_result

In [59]:
_df_ap_modified_2 = _dwt_and_filtering(_df_ap)

In [60]:
_df_ap_modified_2.head()

,timestamp,label,major_label,csi_1,csi_65,csi_2,csi_66,csi_3,csi_67,csi_4,...,csi_119_modified,csi_120_modified,csi_121_modified,csi_122_modified,csi_123_modified,csi_124_modified,csi_125_modified,csi_126_modified,csi_127_modified,csi_128_modified
0,2021-02-09 20:20:17,6,2,84.0,0.0,-64.0,0.0,4.0,-0.145985,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-02-09 20:20:18,6,2,84.0,0.0,-64.0,0.0,4.0,-0.563291,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-02-09 20:20:19,6,2,84.0,0.0,-64.0,0.0,4.0,0.073333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-02-09 20:20:20,6,2,84.0,0.0,-64.0,0.0,4.0,-0.557047,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-02-09 20:20:21,6,2,84.0,0.0,-64.0,0.0,4.0,0.097902,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
col_reindex_modified = ['timestamp', 'label', 'major_label']
for _ in range(64):
    col_reindex_modified.append('csi_{}_modified'.format(_+1))
    col_reindex_modified.append('csi_{}_modified'.format(_+65))

_df_ap_modified_2 = _df_ap_modified_2.reindex(columns=col_reindex_modified)

In [62]:
_df_ap_modified_2.head()

,timestamp,label,major_label,csi_1_modified,csi_65_modified,csi_2_modified,csi_66_modified,csi_3_modified,csi_67_modified,csi_4_modified,...,csi_60_modified,csi_124_modified,csi_61_modified,csi_125_modified,csi_62_modified,csi_126_modified,csi_63_modified,csi_127_modified,csi_64_modified,csi_128_modified
0,2021-02-09 20:20:17,6,2,84.001349,0.0,-63.990894,0.000002,4.00018,-0.498725,0.0,...,-0.505462,0.0,-0.491241,0.0,-0.503089,0.0,-0.480961,0.0,-0.517085,0.0
1,2021-02-09 20:20:18,6,2,84.001349,0.0,-63.990894,0.000002,4.00018,-0.498725,0.0,...,-0.505462,0.0,-0.491241,0.0,-0.503089,0.0,-0.480961,0.0,-0.517085,0.0
2,2021-02-09 20:20:19,6,2,84.001349,0.0,-63.990894,0.000002,4.00018,-0.498725,0.0,...,-0.505462,0.0,-0.491241,0.0,-0.503089,0.0,-0.480961,0.0,-0.517085,0.0
3,2021-02-09 20:20:20,6,2,84.001349,0.0,-63.990894,0.000002,4.00018,-0.498725,0.0,...,-0.505462,0.0,-0.491241,0.0,-0.503089,0.0,-0.480961,0.0,-0.517085,0.0
4,2021-02-09 20:20:21,6,2,84.001349,0.0,-63.990894,0.000002,4.00018,-0.498725,0.0,...,-0.505462,0.0,-0.491241,0.0,-0.503089,0.0,-0.480961,0.0,-0.517085,0.0


 #### <font color="MediumTurquoise"> max-min function</font>
学習main内で使用

In [63]:
def max_min_check(_df):
    max_value = 0
    min_value = 0
    
    col_csi  = _df.columns[3:]
    for col in col_csi:
        max_value_tmp = _df[col].max()
        min_value_tmp = _df[col].min()
        
        max_value = max(max_value, max_value_tmp)
        min_value = min(min_value, min_value_tmp)
    
    return max_value, min_value

## <font color="Orange">  models</font>

 #### <font color="MediumTurquoise">  CNN</font>

In [64]:
def model_cnn(_X_train, _X_test, _y_train):

    _len_train = _X_train.shape
    _X_train = _X_train.values
    _X_train = _X_train.reshape(-1, 8, 8, 2)
    
    _len_test = _X_test.shape
    _X_test = _X_test.values
    _X_test = _X_test.reshape(-1, 8, 8, 2)

    _X_train  = scipy.ndimage.zoom(_X_train , (1,3,3,1))
    _X_test = scipy.ndimage.zoom(_X_test , (1,3,3,1))

    _y_train = np_utils.to_categorical(_y_train, 7)
    
    ###
    learning_rate = 0.001
    epochs = 1000
    batch_size = 64
    ###

    model = Sequential()

    model.add(Conv2D(32, (2, 2), padding='same', input_shape=_X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(7)) 
    model.add(Activation('softmax'))

    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    history = model.fit(_X_train, _y_train, validation_split=0.1, batch_size=batch_size, epochs=epochs)

    _y_test_pred_proba = model.predict(_X_test)
    _y_test_pred = np.argmax(_y_test_pred_proba, axis=1)
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('accuracy')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    
    return _y_test_pred, _y_test_pred_proba

In [65]:
def model_cnn_major(_X_train, _X_test, _y_train):

    _len_train = _X_train.shape
    _X_train = _X_train.values
    _X_train = _X_train.reshape(-1, 8, 8, 2)
    
    _len_test = _X_test.shape
    _X_test = _X_test.values
    _X_test = _X_test.reshape(-1, 8, 8, 2)

    _X_train  = scipy.ndimage.zoom(_X_train , (1,2,2,1))
    _X_test = scipy.ndimage.zoom(_X_test , (1,2,2,1))

    _y_train = np_utils.to_categorical(_y_train, 3)
    
    ###
    learning_rate = 0.0001
    epochs = 1000
    batch_size = 128
    ###

    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=_X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(3)) 
    model.add(Activation('softmax'))

    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    history = model.fit(_X_train, _y_train, validation_split=0.1, batch_size=batch_size, epochs=epochs)

    _y_test_pred_proba = model.predict(_X_test)
    _y_test_pred = np.argmax(_y_test_pred_proba, axis=1)
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('accuracy')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    
    return _y_test_pred, _y_test_pred_proba

 #### <font color="MediumTurquoise">  ResNet</font>

In [66]:
def generate_model(_input_shape, block_f, blocks, block_sets, block_layers=2, first_filters=32, kernel_size=(3,3)):
    inputs = Input(shape=_input_shape)

    # 入力層
    x = Conv2D(filters=first_filters, kernel_size=kernel_size, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D((2, 2))(x)
    x = Dropout(0.2)(x)

    # 畳み込み層
    for s in range(block_sets):
        filters =  first_filters * (2**s)

    for b in range(blocks):
        x = block_f(x, kernel_size, filters, block_layers)

    x = MaxPool2D((2, 2))(x)
    x = Dropout(0.2)(x)

    # 出力層
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(100)(x)
    x = ReLU()(x)
    outputs = Dense(3, activation='softmax')(x)

    model = Model(inputs, outputs)

    return model

In [67]:
def residual_block(x, kernel_size, filters, layers=2):
    shortcut_x = x

    for l in range(layers):
        x = Conv2D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)

        if l == layers-1:
            if K.int_shape(x) != K.int_shape(shortcut_x):
                shortcut_x = Conv2D(filters, (1, 1), padding='same')(shortcut_x)  # 1x1フィルタ

                x = Add()([x, shortcut_x])

        x = ReLU()(x)

    return x

In [68]:
def model_resnet_major(_X_train_valid, _X_test, _y_train_valid):

    _len_train = _X_train_valid.shape
    _X_train_valid = _X_train_valid.values
    _X_train_valid = _X_train_valid.reshape(-1, 8, 8, 2)
    
    _len_test = _X_test.shape
    _X_test = _X_test.values
    _X_test = _X_test.reshape(-1, 8, 8, 2)

    _X_train_valid  = scipy.ndimage.zoom(_X_train_valid , (1,4,4,1))
    _X_test = scipy.ndimage.zoom(_X_test , (1,4,4,1))
    
    _X_train, _X_valid, _y_train,  _y_valid = train_test_split(_X_train_valid, _y_train_valid, test_size=0.1, random_state=0)
    
    _y_train = np_utils.to_categorical(_y_train, 3)
    _y_valid = np_utils.to_categorical(_y_valid, 3)
    
    ###
    learning_rate = 0.0001
    epochs = 500
    batch_size = 128
    ###
    
    _input_shape = _X_train.shape[1:]
    _train_samples = _X_train.shape[0]
    _valid_samples = _X_valid.shape[0]

    model  = generate_model(_input_shape, residual_block, blocks=24, block_sets=2, first_filters=32)

    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10)
    history = model.fit(_X_train, _y_train, epochs=epochs, batch_size=batch_size, validation_data=(_X_valid, _y_valid), callbacks=[early_stopping])

    _y_test_pred_proba = model.predict(_X_test)
    _y_test_pred = np.argmax(_y_test_pred_proba, axis=1)
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('accuracy')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss')
    plt.xlabel('epoch')
    plt.grid()
    plt.legend(['train', 'valid'], loc='upper left')
    plt.show()
    
    return _y_test_pred, _y_test_pred_proba

In [69]:
def model_lgb_major(_X_train, _X_test, _y_train):
    model = lgb.LGBMClassifier(n_jobs=-1, n_estimators=500, class_weight='balanced', random_state=0)
    
    model.fit(_X_train, _y_train)
    
    _y_test_pred = model.predict(_X_test)
    
    _y_test_pred_proba = pd.DataFrame(columns=['proba_0', 'proba_1', 'proba_2'])
    _y_test_pred_proba_0 =  model.predict_proba(_X_test)[:,0]
    _y_test_pred_proba_1 =  model.predict_proba(_X_test)[:,1]
    _y_test_pred_proba_2 =  model.predict_proba(_X_test)[:,2]
#     _y_test_pred_proba_3 =  model.predict_proba(_X_test)[:,3]
#     _y_test_pred_proba_4 =  model.predict_proba(_X_test)[:,4]
#     _y_test_pred_proba_5 =  model.predict_proba(_X_test)[:,5]
#     _y_test_pred_proba_6 =  model.predict_proba(_X_test)[:,6]
    _y_test_pred_proba['proba_0'] = _y_test_pred_proba_0
    _y_test_pred_proba['proba_1'] = _y_test_pred_proba_1
    _y_test_pred_proba['proba_2'] = _y_test_pred_proba_2
#     _y_test_pred_proba['proba_3'] = _y_test_pred_proba_3
#     _y_test_pred_proba['proba_4'] = _y_test_pred_proba_4
#     _y_test_pred_proba['proba_5'] = _y_test_pred_proba_5
#     _y_test_pred_proba['proba_6'] = _y_test_pred_proba_6
    
    _y_test_pred_proba = _y_test_pred_proba.values
     
    return _y_test_pred, _y_test_pred_proba

In [70]:
def model_lgb(_X_train, _X_test, _y_train):
    model = lgb.LGBMClassifier(n_jobs=-1, n_estimators=500, class_weight='balanced', random_state=0)
    
    model.fit(_X_train, _y_train)
    
    _y_test_pred = model.predict(_X_test)
    
    _y_test_pred_proba = pd.DataFrame(columns=['proba_0', 'proba_1', 'proba_2', 'proba_3', 'proba_4', 'proba_5', 'proba_6'])
    _y_test_pred_proba_0 =  model.predict_proba(_X_test)[:,0]
    _y_test_pred_proba_1 =  model.predict_proba(_X_test)[:,1]
    _y_test_pred_proba_2 =  model.predict_proba(_X_test)[:,2]
    _y_test_pred_proba_3 =  model.predict_proba(_X_test)[:,3]
    _y_test_pred_proba_4 =  model.predict_proba(_X_test)[:,4]
    _y_test_pred_proba_5 =  model.predict_proba(_X_test)[:,5]
    _y_test_pred_proba_6 =  model.predict_proba(_X_test)[:,6]
    _y_test_pred_proba['proba_0'] = _y_test_pred_proba_0
    _y_test_pred_proba['proba_1'] = _y_test_pred_proba_1
    _y_test_pred_proba['proba_2'] = _y_test_pred_proba_2
    _y_test_pred_proba['proba_3'] = _y_test_pred_proba_3
    _y_test_pred_proba['proba_4'] = _y_test_pred_proba_4
    _y_test_pred_proba['proba_5'] = _y_test_pred_proba_5
    _y_test_pred_proba['proba_6'] = _y_test_pred_proba_6
    
    _y_test_pred_proba = _y_test_pred_proba.values
     
    return _y_test_pred, _y_test_pred_proba

## <font color="Orange"> train</font>

In [71]:
vc = _df_ap['label'].value_counts()
print(vc)

2    827
6    738
4    674
3    609
1    565
5    527
0    430
Name: label, dtype: int64


In [72]:
vc = _df_ap['major_label'].value_counts()
print(vc)

1    2110
2    1265
0     995
Name: major_label, dtype: int64


In [73]:
def check_accuracy(_df):

    _X = _df.drop(['timestamp','label', 'major_label'], axis=1)
    _y = _df['label']
#     _y = _df['major_label']
    
#     max_value, min_value = max_min_check(_df)
#     _X =  (_X - min_value) / (max_value - min_value)
    
    _X_train, _X_test, _y_train,  _y_test = train_test_split(_X, _y, test_size=0.2, random_state=0)
    
#    _prediction_result = pd.DataFrame(columns=['model', 'data_type', 'nums_test', 'nums_correct', 'nums_ones', 'nums_twos', 'nums_threes', 'acc_vszero', 'confusion_matrix'])
    
    print('start!')
#     _y_test_pred, _y_test_pred_proba = model_cnn(_X_train, _X_test, _y_train)
#     _y_test_pred, _y_test_pred_proba = model_cnn_major(_X_train, _X_test, _y_train)
#     _y_test_pred, _y_test_pred_proba = model_resnet_major(_X_train, _X_test, _y_train)

#     _y_test_pred, _y_test_pred_proba = model_lgb_major(_X_train, _X_test, _y_train)
    _y_test_pred, _y_test_pred_proba = model_lgb(_X_train, _X_test, _y_train)
#     _y_test_pred, _y_test_pred_proba = model_rf_major(_X_train, _X_test, _y_train)

    _y_test = _y_test.values

    nums_test = len(_y_test)
    
    #counts
    _y_diff = np.abs(_y_test - _y_test_pred)
    
    nums_correct = np.count_nonzero(_y_diff == 0)
    nums_ones = np.count_nonzero(_y_diff == 1)
    nums_twos = np.count_nonzero(_y_diff == 2)
    nums_threes = np.count_nonzero(_y_diff == 3)
    nums_fours = np.count_nonzero(_y_diff == 4)
    nums_fives = np.count_nonzero(_y_diff == 5)
    nums_sixs = np.count_nonzero(_y_diff == 6)
    
#     #zero vs non-zero
#     _y_test_vszero = np.where(_y_test < 1, 0, 1)
#     _y_test_pred_vszero = np.where(_y_test_pred < 1, 0, 1)
 
#     acc= accuracy_score(_y_test_vszero, _y_test_pred_vszero)
    
    print('■accuracy')
    print('acc : {}'.format(round((nums_correct/nums_test), 4)))
    print('\n')
    
    #counts
    print('■counts [all : {}]'.format(nums_test))
    print('corret : {} [{}%]'.format(nums_correct, round((nums_correct/nums_test)*100)))
    print('non-correct +1 or -1 : {} [{}%]'.format(nums_ones, round((nums_ones/nums_test)*100)))
    print('non-correct +2 or -2 : {} [{}%]'.format(nums_twos, round((nums_twos/nums_test)*100)))
    print('non-correct +3 or -3 : {} [{}%]'.format(nums_threes, round((nums_threes/nums_test)*100)))
    print('non-correct +4 or -4 : {} [{}%]'.format(nums_fours, round((nums_fours/nums_test)*100)))
    print('non-correct +5 or -5 : {} [{}%]'.format(nums_fives, round((nums_fives/nums_test)*100)))
    print('non-correct +6 or -6 : {} [{}%]'.format(nums_sixs, round((nums_sixs/nums_test)*100)))

    #zero vs non-zero
#     print('■zero vs non-zero') 
#     print('accuracy : {}'.format(round(_prediction_result_test['acc_vszero'][0],3)))
    
#     return _prediction_result
    return

In [74]:
def main(_df):
    check_accuracy(_df)
    
#     evaluation_output(_prediction_result)
    
#     return _prediction_result
    return 0

In [75]:
# _prediction_result = main(_df_ap)

In [76]:
_prediction_result = main(_df_mp)

start!
■accuracy
acc : 0.953


■counts [all : 1000]
corret : 953 [95%]
non-correct +1 or -1 : 22 [2%]
non-correct +2 or -2 : 23 [2%]
non-correct +3 or -3 : 2 [0%]
non-correct +4 or -4 : 0 [0%]
non-correct +5 or -5 : 0 [0%]
non-correct +6 or -6 : 0 [0%]


In [77]:
_prediction_result = main(_df_ap_modified)

start!
■accuracy
acc : 0.8432


■counts [all : 874]
corret : 737 [84%]
non-correct +1 or -1 : 53 [6%]
non-correct +2 or -2 : 55 [6%]
non-correct +3 or -3 : 18 [2%]
non-correct +4 or -4 : 8 [1%]
non-correct +5 or -5 : 3 [0%]
non-correct +6 or -6 : 0 [0%]


In [78]:
_prediction_result = main(_df_ap_modified_2)

start!
■accuracy
acc : 0.4989


■counts [all : 874]
corret : 436 [50%]
non-correct +1 or -1 : 193 [22%]
non-correct +2 or -2 : 119 [14%]
non-correct +3 or -3 : 90 [10%]
non-correct +4 or -4 : 28 [3%]
non-correct +5 or -5 : 8 [1%]
non-correct +6 or -6 : 0 [0%]


In [ ]:
_prediction_result = main(_df_ap)

start!
